In [4]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import gym

In [8]:
env = gym.make('Pendulum-v1').unwrapped

In [10]:
state_shape = env.observation_space.shape[0]
action_shpae = env.action_space.shape[0]
action_bound = [env.action_space.low, env.action_space.high]
epsilon = 0.2

In [1]:
class PPO(object):
    def __init__(self):
        # start the tensorFlow session
        self.sess = tf.Session()
        # Define the placeholder for the state
        self.state_ph = tf.placeholder(tf.float32, [None, state_shape],'state')
        # build the value network that returns the value of a state
        with tf.variable_scope('value'):

            layer1 = tf.layers.dense(self.state_ph, 100, tf.nn.relu)
            self.v = tf.layers.dense(layer1, 1)
            self.Q = tf.placeholder(tf.float32, [None,1], 'discounted_r')
            self.advantage = self.Q - self.v
            self.value_loss = tf.reduce_mean(tf.square(self.advantage))
            self.train_value_nw = tf.train_AdamOptimizer(0.002).minimize(self.value_loss)
        
        pi, pi_params = self.build_policy_network('pi', trainable = True)
        oldpi, oldpi_params = self.build_policy_network('oldpi', trainable = False)

        with tf.variable_scope('sample_action'):
            self.sample_op = tf.squeeze(pi.sample(1), axis = 0)

        with tf.variable_scope('update_oldpi'):
            self.update_oldpi_op = [oldp.assign(p) for p, oldp in zip(pi_params, oldpi_params)]

        self.action_ph = tf.placehoder(tf.float32, [None, action_shpae], 'action')
        self.advantage_ph = tf.placehoder(tf.float32, [None, 1], 'advantage')

        with tf.variable_scope('loss'):
            with tf.variable_scope('surrogate'):
                ratio = pi.prob(self.action_ph) / oldpi.prob(self.action_ph)
                objective = ratio * self.advantage_ph
                L = tf.reduce_mean(tf.minimum(objective, tf.clip_by_value(ratio, 1.-epsilon,1.+epsilon)*self.advantage_ph))
            self.policy_loss = -L

        with tf.variable_scope('train_policy'):
            self.train_policy_nw = tf.train_AdamOptimizer(0.001).minimize(self.policy_loss)

        self.sess.run(tf.global_variables.initializer())

    def train(self, state, action, reward):
        self.sess.run(self.update_oldpi_op)
        adv = self.sess.run